# Notebook Dedicated to Obtain an Estimative for the total Live-time of the Used Dataset

In [6]:
import uproot
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sn

import glob
import re
import os
import itertools
import pickle

# Using the LT List of Daniel

## Load Data

### List of the Run Numbers of the Dataset

In [7]:
main_dir = 'E:/Data/solars/solarnu_Realdata/'
fname = 'run_numbers.txt'

runID_list =[]

with open(main_dir + fname, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            run_id_i = line.split()
            runID_list.append(int(run_id_i[0]))
        
runID_list = np.array(runID_list)
runID_list = np.unique (runID_list)

In [8]:
runID_list

array([300007, 300023, 300436, 300487, 300500, 300817, 300854, 300901,
       300944, 300947, 301011, 301074, 301131, 301144, 301232, 301304,
       301335, 301351, 301365, 301418, 301423, 301475, 301480, 301494,
       301495, 301504, 301649, 301653, 301721, 301973, 302005, 303039,
       303109, 303443, 303858, 303878, 304022, 304091, 304095, 304120,
       304159, 304171, 304191, 304199, 304867, 304896, 304914, 305087,
       305366, 305386, 305495, 305581, 305600, 305708, 305953, 306362,
       306424, 306425, 306427, 306522, 306670, 306778, 306975, 306984,
       307005, 307055, 307068, 307115, 307153, 307409, 307458, 307464,
       307497, 307550, 307625, 307626, 307632, 307650, 307848, 307932,
       307976, 308010, 308024, 308027, 308032, 308091, 308135, 308160,
       308243, 308722, 308825, 308852, 308937, 308944, 308948, 309042,
       309233])

In [9]:
runID_list.shape

(97,)

### List of the Run Numbers with LT values

In [10]:
file_dir = 'E:/Data/antinu/antinu_realdata/runID_list_time.txt'

runID_lt, time1_lt, time2_lt = [], [], []

with open(file_dir, "r", encoding="utf-8") as f:
    for linea in f:
        if linea.strip():  # Evitar líneas vacías
            valores = linea.split()  # Separa por espacios o tabulaciones
            runID_lt.append(int(valores[0]))
            time1_lt.append(float(valores[1]))
            time2_lt.append(float(valores[2]))
            
runID_lt = np.array(runID_lt)
time2_lt = np.array(time2_lt) # NET LIVETIME (DAYS)

In [11]:
av_run_time = np.mean(time2_lt) # Avarage Run time. Is observed the Lack of a run ID in the live-time list
                                # Take the Run Time avarage to fill this values
av_run_time

0.04045685096405452

## Livetime Calculation

In [12]:
# Extract the index of runID_lt where the values of the dataset runID_list exist
index_lt = np.where((np.in1d(runID_lt, runID_list)))[0]

tot_lt_days = np.sum(time2_lt[index_lt]) 
tot_lt_days + av_run_time

3.9804650983020173

In [13]:
tot_lt_days

3.9400082473379627

In [71]:
np.max(runID_list)

309233

In [72]:
index_lt.shape

(96,)

In [76]:
np.unique(runID_lt).shape

(6099,)

In [99]:
time = []
for run_i in np.unique(runID_list):
    run_id_condition = (runID_lt == run_i)
    selec_time = time2_lt[run_id_condition]
    time.append(selec_time)

In [96]:
runID_lt

array([300000, 300001, 300002, ..., 310635, 310636, 310637])

In [94]:
runID_lt[np.where((np.in1d(runID_lt, runID_list)))[0]]

array([300007, 300023, 300436, 300487, 300500, 300817, 300854, 300901,
       300944, 300947, 301011, 301074, 301131, 301144, 301232, 301304,
       301335, 301351, 301365, 301418, 301423, 301475, 301480, 301494,
       301495, 301504, 301649, 301653, 301721, 301973, 302005, 303039,
       303109, 303443, 303858, 303878, 304091, 304095, 304120, 304159,
       304171, 304191, 304199, 304867, 304896, 304914, 305087, 305366,
       305386, 305495, 305581, 305600, 305708, 305953, 306362, 306424,
       306425, 306427, 306522, 306670, 306778, 306975, 306984, 307005,
       307055, 307068, 307115, 307153, 307409, 307458, 307464, 307497,
       307550, 307625, 307626, 307632, 307650, 307848, 307932, 307976,
       308010, 308024, 308027, 308032, 308091, 308135, 308160, 308243,
       308722, 308825, 308852, 308937, 308944, 308948, 309042, 309233])

# Using the Universal Time

The Idea is to read the Utime (days, secs, nsecs) of events and perform a diference between consecutive different values for different events of UTime. These differences are then added and we get and effective lifetime of the real dataset

## Load Data

In [4]:
# ========= Create file Directory List =========
main_dir = 'E:/Data/solars/solarnu_Realdata/energy_down_2_pt_5_MeV/Bi214_coinc_2.5MeV/'
fname_pattern = 'real_data_solar_analysis_*.root'

flist = glob.glob(os.path.join(main_dir, fname_pattern))

def alpha_num_sort(list_route):
    def extract_num(route):
        filename = os.path.basename(route)
        match = re.search(r'analysis_(\d+)\.root$', filename)
        return int(match.group(1)) if match else -1
    return sorted(list_route, key=extract_num)

flist = alpha_num_sort(flist)
flist

['E:/Data/solars/solarnu_Realdata/energy_down_2_pt_5_MeV/Bi214_coinc_2.5MeV\\real_data_solar_analysis_0.root',
 'E:/Data/solars/solarnu_Realdata/energy_down_2_pt_5_MeV/Bi214_coinc_2.5MeV\\real_data_solar_analysis_1.root',
 'E:/Data/solars/solarnu_Realdata/energy_down_2_pt_5_MeV/Bi214_coinc_2.5MeV\\real_data_solar_analysis_2.root',
 'E:/Data/solars/solarnu_Realdata/energy_down_2_pt_5_MeV/Bi214_coinc_2.5MeV\\real_data_solar_analysis_3.root',
 'E:/Data/solars/solarnu_Realdata/energy_down_2_pt_5_MeV/Bi214_coinc_2.5MeV\\real_data_solar_analysis_4.root',
 'E:/Data/solars/solarnu_Realdata/energy_down_2_pt_5_MeV/Bi214_coinc_2.5MeV\\real_data_solar_analysis_5.root',
 'E:/Data/solars/solarnu_Realdata/energy_down_2_pt_5_MeV/Bi214_coinc_2.5MeV\\real_data_solar_analysis_6.root',
 'E:/Data/solars/solarnu_Realdata/energy_down_2_pt_5_MeV/Bi214_coinc_2.5MeV\\real_data_solar_analysis_7.root',
 'E:/Data/solars/solarnu_Realdata/energy_down_2_pt_5_MeV/Bi214_coinc_2.5MeV\\real_data_solar_analysis_8.root',
 

## One File Proof

In [24]:
# ========= Read information from the file list =========
file = uproot.open(flist[0])
data = file['T;1']

evtid = np.array(data['evtid'])
ev_time_day = np.array(data['ev_time_day'])
ev_time_sec = np.array(data['ev_time_sec'])  # -> Pass to days through sec/86400
ev_time_nanosec = np.array(data['ev_time_nanosec'])# -> Pass to days through nsec/(1e9*86400)

# ===== Remove  redundante values through evID =====
#Empty lists tyo be filled with the non-repeated event values
evtid_ev = []
ev_time_day_ev = []
ev_time_sec_ev = []
ev_time_nanosec_ev = []

#evID index break list
data_break_i = [0]  #array whose elements are the index from where an event start and end.

N_data = len(evtid)

for i_dx in range(N_data - 1):
    if evtid[i_dx] != evtid[i_dx+1]:
        data_break_i.append(i_dx+1)

#add the last index to complete the data_break_i list
data_break_i.append(N_data - 1)

N_terms = len(data_break_i)

#Start to save the non-redundant values and separate the time residuals and angles for each event
for i_dx in range(N_terms - 2):
    init_i = data_break_i[i_dx]
    final_i = data_break_i[i_dx+1]

    evtid_ev.append(evtid[init_i: final_i][0])
    ev_time_day_ev.append(ev_time_day[init_i: final_i][0])
    ev_time_sec_ev.append(ev_time_sec[init_i: final_i][0])
    ev_time_nanosec_ev.append(ev_time_nanosec[init_i: final_i][0])

evtid_ev = np.array(evtid_ev)
ev_time_day_ev = np.array(ev_time_day_ev)
ev_time_sec_ev = np.array(ev_time_sec_ev)
ev_time_nanosec_ev = np.array(ev_time_nanosec_ev)

# ===== Live-time Computation =====

#Time diferences:
dt_days = np.diff(ev_time_day_ev)
dt_sec = np.diff(ev_time_sec_ev)
dt_nsec = np.diff(ev_time_nanosec_ev)

#Total Times:
days_tot = np.sum(dt_days)
sec_tot = np.sum(dt_sec)
nsec_tot = np.sum(dt_nsec)

live_time_tot = days_tot + sec_tot/86400.0 + nsec_tot/(1.0e9 * 86400.0)
live_time_tot

9.804998120644676

In [5]:
flist

['E:/Data/solars/solarnu_Realdata/energy_down_2_pt_5_MeV/Bi214_coinc_2.5MeV\\real_data_solar_analysis_27.root',
 'E:/Data/solars/solarnu_Realdata/energy_down_2_pt_5_MeV/Bi214_coinc_2.5MeV\\real_data_solar_analysis_28.root']

## All Files UTime Reading

In [30]:
# Save the total interval times for each readen file
days_tot = []
sec_tot = []
nsec_tot = []

for index_file, input_file_dir in enumerate(flist):
    #print(f'In file {input_file_dir}')
    
    #Load the Data:
    load_data = uproot.open(input_file_dir)

    #select the tree of event data
    TTree_data_name = load_data.keys()[0]
    event_data = load_data[TTree_data_name]

    #Select observables
    evtid = np.array(event_data['evtid'])
    ev_time_day = np.array(event_data['ev_time_day'])
    ev_time_sec = np.array(event_data['ev_time_sec'])  # -> Pass to days through sec/86400
    ev_time_nanosec = np.array(event_data['ev_time_nanosec'])# -> Pass to days through nsec/(1e9*86400)

    # ===== Remove  redundante values through evID =====
    #Empty lists tyo be filled with the non-repeated event values
    evtid_ev = []
    ev_time_day_ev = []
    ev_time_sec_ev = []
    ev_time_nanosec_ev = []
    
    #evID index break list
    data_break_i = [0]  #array whose elements are the index from where an event start and end.
    
    N_data = len(evtid)
    for i_dx in range(N_data - 1):
        if evtid[i_dx] != evtid[i_dx+1]:
            data_break_i.append(i_dx+1)
    
    #add the last index to complete the data_break_i list
    data_break_i.append(N_data - 1)
    
    N_terms = len(data_break_i)
    
    #Start to save the non-redundant values and separate the time residuals and angles for each event
    for i_dx in range(N_terms - 2):
        init_i = data_break_i[i_dx]
        final_i = data_break_i[i_dx+1]
    
        evtid_ev.append(evtid[init_i: final_i][0])
        ev_time_day_ev.append(ev_time_day[init_i: final_i][0])
        ev_time_sec_ev.append(ev_time_sec[init_i: final_i][0])
        ev_time_nanosec_ev.append(ev_time_nanosec[init_i: final_i][0])
    
    evtid_ev = np.array(evtid_ev)
    ev_time_day_ev = np.array(ev_time_day_ev)
    ev_time_sec_ev = np.array(ev_time_sec_ev)
    ev_time_nanosec_ev = np.array(ev_time_nanosec_ev)
    
    # ===== Live-time Computation =====
    #print('Live-time Calculation ...')
    
    #Time diferences:
    dt_days = np.diff(ev_time_day_ev)
    dt_sec = np.diff(ev_time_sec_ev)
    dt_nsec = np.diff(ev_time_nanosec_ev)
    
    #Save Total Times per file:
    days_tot.append(np.sum(dt_days))
    sec_tot.append(np.sum(dt_sec))
    nsec_tot.append(np.sum(dt_nsec))

#Total Live-time Calculation:
days_tot = np.array(days_tot)
sec_tot = np.array(sec_tot)
nsec_tot = np.array(nsec_tot)

live_time_tot = np.sum(days_tot) + np.sum(sec_tot)/86400.0 + np.sum(nsec_tot)/(1.0e9 * 86400.0)
print(f'Total Live-time = {live_time_tot}')

Total Live-time = 308.79705514596526
